数据集：https://github.com/fatecbf/toutiao-text-classfication-dataset/blob/master/toutiao_cat_data.txt.zip

搜索引擎的三个步骤：
- 自动下载网页(网页爬虫)
- 建立索引
- 度量网页质量(pagerank)

In [1]:
data = open('./toutiao_cat_data.txt',encoding='utf-8').readlines()

In [2]:
from collections import Counter
import pandas as pd
import jieba
import numpy as np

In [3]:
data[1]

'6552368441838272771_!_101_!_news_culture_!_发酵床的垫料种类有哪些？哪种更好？_!_\n'

In [4]:
# 将每行的字符串切割
data = [d.split('_!_') for d in data]

In [5]:
# 根据切割后的数据，创建 DataFrame
# 新闻id，分类code，分类名称，新闻字符串，新闻关键词
df = pd.DataFrame(data, columns=['id', 'code', 'category', 'content', 'keys'])

In [6]:
# 跑不动就减小数据集
df = df[:50000]

In [7]:
# 数据集大小 
NUM_DOCUMENTS = len(df)
df.head()

,id,code,category,content,keys
0,6551700932705387022,101,news_culture,京城最值得你来场文化之旅的博物馆,"保利集团,马未都,中国科学技术馆,博物馆,新中国\n"
1,6552368441838272771,101,news_culture,发酵床的垫料种类有哪些？哪种更好？,\n
2,6552407965343678723,101,news_culture,上联：黄山黄河黄皮肤黄土高原。怎么对下联？,\n
3,6552332417753940238,101,news_culture,林徽因什么理由拒绝了徐志摩而选择梁思成为终身伴侣？,\n
4,6552475601595269390,101,news_culture,黄杨木是什么树？,\n


# 进行分词
[结巴中文分词](https://github.com/fxsjy/jieba)

In [8]:
# 使用 jieba 库将 content 列的每行进行中文分词，之后填回原处
# 使用空格连接分出来的词，组成一个字符串
df['content'] = df.content.apply(lambda x: ' '.join(jieba.cut(x)))
df.head()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\sunsh\AppData\Local\Temp\jieba.cache
Loading model cost 0.736 seconds.
Prefix dict has been built succesfully.


,id,code,category,content,keys
0,6551700932705387022,101,news_culture,京城 最 值得 你 来场 文化 之旅 的 博物馆,"保利集团,马未都,中国科学技术馆,博物馆,新中国\n"
1,6552368441838272771,101,news_culture,发酵 床 的 垫料 种类 有 哪些 ？ 哪 种 更好 ？,\n
2,6552407965343678723,101,news_culture,上联 ： 黄山 黄河 黄皮肤 黄土高原 。 怎么 对 下联 ？,\n
3,6552332417753940238,101,news_culture,林徽因 什么 理由 拒绝 了 徐志摩 而 选择 梁思成 为 终身伴侣 ？,\n
4,6552475601595269390,101,news_culture,黄杨木 是 什么 树 ？,\n


In [14]:
df.tail()

,id,code,category,content,keys
49995,6538297822440063501,101,news_culture,“ 曾经沧海 难为 水 ， 除却 巫山 不是 云 ” 背后 的 现实 与 理想,"巫山,莺莺传,官宦人家,曾经沧海难为水,崔莺莺,元稹,西厢记,离思五首其四\n"
49996,6494757116576792845,101,news_culture,战国 水晶 杯 是 有人 穿越 带 过去 的 吗 ？,"石塘村,战国水晶杯,春秋时期,青铜器,战国时期\n"
49997,6534277630332502532,101,news_culture,《 水浒传 》 中 4 个 虚张声势 的 好汉 ， 听 绰号 以为 是 英雄 ， 实际 是 讽刺,"百胜将,方腊,水浒传,杜迁,韩滔,梁山\n"
49998,6530928677306237454,101,news_culture,奇葩 汉字 盘点 ， 你 认得 几个 ？,"tǔ,kāo,引申义,dá,后汉书·侯霸传,尾椎骨,玉篇\n"
49999,6547230706886508803,101,news_culture,八十年代 的 庞 中华 字帖 ， 现在 销声匿迹 了 吗 ？ 为什么 ？,\n


In [20]:
# extend 将元素从新列表中取出，放入原列表中
a = ['我','你']
b= ['好','吗']
a.append(b)
print(a)
a = ['我','你']
a.extend(b)
print(a)

['我', '你', ['好', '吗']]
['我', '你', '好', '吗']


In [21]:
# 单词计数
obj = Counter(a)
print(obj)

Counter({'我': 1, '你': 1, '好': 1, '吗': 1})


In [25]:
# enumerate 遍历一个对象并添加索引
# 遍历一个字典实际上遍历的是字典的keys
a_vocab = dict(Counter(a))
a_word2id = {k:v for v,k in enumerate(a_vocab.keys())}
print(a_word2id)
a_word2id = {k:v for v,k in enumerate(a_vocab)}
print(a_word2id)

{'我': 0, '你': 1, '好': 2, '吗': 3}
{'我': 0, '你': 1, '好': 2, '吗': 3}


In [61]:
# 创建字典
vocab = []
# extend 将元素从新列表中取出，放入原列表中
for document in df.content:
    words = document.split()
    vocab.extend(words)

# 总单词数目
NUM_TREMS = len(vocab)
# 词汇表 - 单词计数
vocab = dict(Counter(vocab))
# 根据词频排序
vocab = sorted(vocab.items(), key=lambda x:x[1], reverse=True)
# 转换成字典
vocab = dict(vocab)

# 过滤出现次数少的单词：不重要的单词（或拼写错误的单词）
# 去掉少于5000=>100(只取部分数据时)词频的单词
print(len(vocab))
vocab = {k:v for k,v in vocab.items() if v>100}
print(len(vocab))

# word2id id2word
word2id = {k:v for v,k in enumerate(vocab.keys())}
id2word = {v:k for k,v in word2id.items()}
# 字典数量
NUM_WORDS = len(vocab)
print('字典中单词的种类：',NUM_WORDS) # vocab.keys(), word2id
print('字典中所有单词的数量',NUM_TREMS)

55030
693
字典中单词的种类： 693
字典中所有单词的数量 658540


# 创建索引

最简单的索引就是对每一个文档(网页)创建一个很长的二进制数字，每个数字表示其对应的关键字是否在文档当中。

二维数组
``` 
way1

         文档1  文档2 ...
关键字1    1      0
关键字2    0      1
....
```

In [34]:
# 一维数组
# 判断字典 word2id 中每个单词是否在字符串 x 中
def generate_index(x):
    """
    将字符串转换成id list
    """
    index = [0] * NUM_WORDS
    words = x.split()
    for word in words:
        i = word2id.get(word, -1)
        if i == -1: continue
        index[i] = 1
    return index

# 验证一下是否是想要的结果
r = generate_index('京城 最 值得 你 来场 文化 之旅 的 博物馆')
# r


> ps: 需要运行很长时间，占用很大内存条

In [38]:
# 正向索引：二维索引
# 判断字典中的每个单词是否在 df.content 的每行文本中
all_index_zh = df.content.apply(generate_index)
print(np.array(all_index_zh.tolist()).shape)

(50000, 693)


``` 
way2
       关键字1  关键字2   关键字3 ...
文档1    1        0       0
文档2    0        1       1
...
```
将其转置一下就可以了
``` 
way3
        文档1   文档2    文档3 ...
关键字1   1       0       0
关键字2   0       1       1
...
```

In [13]:
# 倒排索引：
all_index = np.array(all_index_zh.tolist()).T
print(all_index.shape)

(693, 50000)

In [26]:
all_index[0:5]

array([[0, 0, 0, ..., 1, 1, 1],
       [0, 1, 1, ..., 0, 1, 1],
       [1, 1, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

这样会生成一个表，即索引表：表的每一行对应一个关键字，而每一个关键字后面跟着一组数字，表示哪些文档包含这个关键字。  
这时候，当我们查询某个关键字的时候，不再需要将整个文档集合从头到尾查看一遍，只需要查找这个表就知道应该取哪些文档中去搜索就可以了。  
这和我们生活中的书籍的目录是非常相似的。  

当我们想要查询多个关键字的文档时候，也只需要将两个关键字对应的索引向量进行一个布尔运算(and)，便能将所有出现两个关键字的文档检索出来。  
计算机对于布尔运算的计算速度是非常快的。

In [49]:
row1 = all_index[word2id['北京']]
row2 = all_index[word2id['房价']]
print(row1)
print(row2)
# 逐位与，查找同时出现'北京'和'房价'关键词的所有文档
row3 = (row1 & row2)
search_res = []
for i,word in enumerate(row3):
    if word==1:
        search_res.append(i)
print('同时出现`北京`和`房价`关键词的所有文档：',search_res)

[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
同时出现`北京`和`房价`关键词的所有文档： [11848, 21857, 23181, 23516, 25454, 28720, 29154, 45713]


## 大数据

假如互联网有$100亿^3$($10^{10}$)个有用的网页，再假设词汇表有30万个单词(肯定偏小)，那么索引的大小至少是100亿x30万=3000万亿。  
假如我们去掉一些只出现一个网页(文档)中的词，取百分之一，那也有30万亿。  

不仅这样，索引除了保存关键字信息，还要有一些其他信息需要保存，比如词出现的位置啦，次数啦，  
这些数据一台服务器是村放不下的，所以通常把索引切片，放到不同服务器上，这就和服务器的集群，分布式很相关了。

将数据存到不同的服务器上，就会引申出很多其它问题，比如，如何保证数据的一致性(数据脏读等等)，如何降低服务期间的通信成本，否则分布式效率还不如一台服务器效果高。

总之，由搜索引擎的索引可以扩展到很多很深的知识，这些知识偏向工程化，但工程化的问题不管多么复杂，至少在索引的原理上，还是非常简单的。类似目录，等价于布尔运算。

# 度量网页质量

Google的pagerank方法是常用的网页质量度量方法，在NLP领域中也是比较重要的一个方法。由于此数据集不适用于pagerank方法，这里就不再展开。  
仅做简单介绍，有兴趣的同学可自行深入理解研究，或在开课吧NLP课程中进行学习。

在掌握
- 数据下载
- 建立索引
- 度量网页

后，我们再来看下搜索引擎的其它考量：
1. 索引要完善，巧妇难为无米之炊，算法再好，要找的答案不再候选答案集合中，那都是徒劳的。
2. 网页质量的度量，一些八卦性质的网站pagerank值比较高，但权威性质就比较低
3. 用户偏好，不同的用户对结果的评判标准不一样，个性化推荐(深入就是走向推荐的方向了)
4. 确定查询和网页的相关性

OK，我们来看第4条：怎么确定网页和查询的相关性的度量

# 确定网页和查询的相关性

我们常用词频(Term Frequency)的概念来表示：  
$$tf = \frac{单词的出现频率}{网页的总字数}$$

所以，一个文档中的单词出现的词频可以表示为：$TF_1, TF_2, ...$  
那么假如给定关键字$w_1, w_2$，可以计算每个文档中两个关键字的词频，记为$TF_1, TF_2$  

比如 `北京` `首都`   在文档 `可爱 的 北京 是 祖国 的 首都， 北京 是 个 美丽 的 城市` 和 文档`他 喊道 北京 北京 我爱你， 北京 北京 我 来了， 首都 北京 我 来 了`
词频分别是
[2/13, 1/13]
[5/14, 1/14]

所以，我们可以把查询关键字和文档的相关性的度量表示为：
$TF_1+TF_2$

当查询q有N个关键字，M个文档时候，第i个文档与查询q的相似度可以记为：

$$sim(q, d_i) = TF_1+TF_2+....TF_N, (q=w_1,w_2,...w_n)$$

In [51]:
# 过滤掉查询关键字不在字典中的词

query = '北京的房价多少？'
# query = '特朗普是什么的？'
query = list(jieba.cut(query))

# 过滤掉不在字典中的词
[word for word in query if word in vocab]

['北京', '的', '房价', '多少', '？']

计算每个关键字的TF

In [56]:
def querybykeys(query):
    finds = np.array([1] * NUM_DOCUMENTS) # NUM_DOCUMENTS = len(df) 数据集的大小 
    for key in query:
        finds += all_index[word2id[key]] # 利用索引快速找到包含当前关键字的数据
        finds = finds==2
        finds = finds.astype(int)
    return finds

def querybykeys_2(query):
    finds = np.array([1] * NUM_DOCUMENTS) # NUM_DOCUMENTS = len(df) 数据集的大小 
    for key in query:
        finds &= all_index[word2id[key]] # 利用索引快速找到包含当前关键字的数据
    return finds

query = ['北京','房价']
finds = querybykeys(query)
finds = [i for i,j in enumerate(finds) if j]
print('Find document id that contains all keys:',finds)

finds = querybykeys(query)
finds = [i for i,j in enumerate(finds) if j]
print('Find document id that contains all keys - 2:',finds)

Find document id that contains all keys: [11848, 21857, 23181, 23516, 25454, 28720, 29154, 45713]
Find document id that contains all keys - 2: [11848, 21857, 23181, 23516, 25454, 28720, 29154, 45713]


In [55]:
# 既然没有那咱找几个小的数据进行继续的说明
finds = querybykeys(['北京', '房价']) # ['北京', '房价']
finds = [i for i,j in enumerate(finds) if j]
print('Find document id that contains all keys:', finds)

# 将这些数据打印出来
for d in finds:
    print(df.iloc[d].content)

Find document id that contains all keys: [11848, 21857, 23181, 23516, 25454, 28720, 29154, 45713]
房价 白跌 了 ！ 北京 首 套房 贷款 利率 上浮 10%
你 认为 北京 未来 5 年 的 房价 会 下跌 吗 ？
「 重磅 」 北京 限 房价 项目 销售 办法 出炉   部分 将 收作 共有 产权 房
北京 的 房价 还会 继续 上涨 吗 ？
北京 房贷利率 下周 再涨   专家 ： 累计 相当于 房价 上调 了 10%
北京 限价 房转 共有 产权 房 政策 拟 出炉   对 房价 有何 影响 ？
房价 不涨 卖 不动 ， 北京 首 套房 利率 继续 上浮 ， 刚需 招惹 了 谁 ？
北京 的 房价 还会 继续 上涨 吗 ？


In [59]:
# 计算每个数据的TF
tf_result = []
for d in finds:
    words = df.iloc[d].content.split()
    # 统计单词出现次数
    tf = dict(Counter(words))
    # 统计词频
    tf = {k:v/len(tf) for k,v in tf.items() if k in query}
    # 统计query的词频
    tf_result.append(sum(tf.values()))
    # 调试打印
    print(words)
    print(tf)
    print(sum(tf.values()))
    print()
    
    
# 返回所有的权重结果
# 找到最大的数据对应的index
i = np.argmax(tf_result)
# 打印检索后的结果
print(f'tf:{max(tf_result)} 检索结果:{df.iloc[finds[i]].content}')

['房价', '白跌', '了', '！', '北京', '首', '套房', '贷款', '利率', '上浮', '10%']
{'房价': 0.09090909090909091, '北京': 0.09090909090909091}
0.18181818181818182

['你', '认为', '北京', '未来', '5', '年', '的', '房价', '会', '下跌', '吗', '？']
{'北京': 0.08333333333333333, '房价': 0.08333333333333333}
0.16666666666666666

['「', '重磅', '」', '北京', '限', '房价', '项目', '销售', '办法', '出炉', '部分', '将', '收作', '共有', '产权', '房']
{'北京': 0.0625, '房价': 0.0625}
0.125

['北京', '的', '房价', '还会', '继续', '上涨', '吗', '？']
{'北京': 0.125, '房价': 0.125}
0.25

['北京', '房贷利率', '下周', '再涨', '专家', '：', '累计', '相当于', '房价', '上调', '了', '10%']
{'北京': 0.08333333333333333, '房价': 0.08333333333333333}
0.16666666666666666

['北京', '限价', '房转', '共有', '产权', '房', '政策', '拟', '出炉', '对', '房价', '有何', '影响', '？']
{'北京': 0.07142857142857142, '房价': 0.07142857142857142}
0.14285714285714285

['房价', '不涨', '卖', '不动', '，', '北京', '首', '套房', '利率', '继续', '上浮', '，', '刚需', '招惹', '了', '谁', '？']
{'房价': 0.0625, '北京': 0.0625}
0.125

['北京', '的', '房价', '还会', '继续', '上涨', '吗', '？']
{'北京': 0.125, '房价': 0.12

这里面大家有没有发现一个问题？  
因为数据量比较少，可能不太明显，  
但你还是能够感受得到的。

在sim最大的两条数据中，`北京` `房价` `的` 三个单词分别贡献了0.125的权重  
但从直观理解来说，`的`字对于问题其实完全没有帮助,因为对于任何问题基本都有`的`字，并且文档中`的`字的频次会更高，这对问答的匹配来说是无效，甚至有害的。

## 逆文本频率 （Inverse Document Frequency）
**TF-IDF**

那么咱们来将我们需要的单词更加明确化一些。  
当然，最直接的方法是使用停用词表，将常用的无意义的单词如'的','了'等放入一个词汇表，在检索的关键词中过滤掉这些单词。

但这个方法有一定的限制，最简单的停用词表是基本没有问题的。但去掉这些后，对我们检索的结果提升并不会非常大。另外，不同的领域可能拥有不同的停用词，如果对每个任务都手工去
创建停用词表，是非常费时费力的。

那么有没有一种方法能够帮助我们实现这个功能呢？  

其实，我们可以设定两个条件：
1. 一个词预测主题的能力越强，那么它对应的权重应该更大，反之越小。比如 `房价`能帮助我们或多或少的了解当前文档的主题。'多少'就不行，那么`房价`的权重应该大于`多少`
2. 停用词的权重为零

即假定一个关键词$w$在$D_w$个网页中出现过，那么$D_w$越大，$w$的权重越小，反之亦然。  
这就是 **逆文本频率** （Inverse Document Frequency，简写为IDF）  
$$IDF = \log \frac{D}{D_w}$$
$D$是全部文档数量。  
比如全部文档是1亿，每个文档中都出现了`的`字，那么它的IDF = log(1亿/1亿）=log 1= 0  
加入`房价`只出现了20万个网页中，那么它的IDF = log(500) = 8.96  
而'多少'出现了5000W个文档中，它的权重IDF = log(2) = 1。

所以最终的权重由之前的词频相加变成了：  
$$TF_1\cdot IDF_1 + TF_2\cdot IDF_2+ ... +TF_N\cdot IDF_N$$

当然，如果你使用了网页排名(pagerank)，那么查询的结果可以由相关性和网页排名的成绩决定。

In [60]:
# 计算单词的IDF

# 语料 - 所有文档的所有单词
corpus = []
for document in df.content:
    corpus.append(document.split())

创建IDF，以后使用直接调用

In [81]:
idf = {}
for word in vocab:
    count = 0
    for line in corpus:
        if word in line:
            count += 1
    idf[word] = round(np.log(NUM_TREMS/count),4) # D/Dw
    # NUM_TREMS = len(vocab) 总单词数目


In [82]:
# 排序简单看下权重大的单词
idf_min = sorted(idf.items(), key=lambda x:x[1]) # 反向 即权重最小的
print(idf_min[:5]) 

[('？', 3.1594), ('，', 3.1902), ('的', 3.6054), ('！', 4.5247), ('：', 4.6072)]


In [83]:
idf_max = sorted(idf.items(), key=lambda x:x[1], reverse=True) # 正向 权重最大的
print(idf_max[:5]) 

[('五', 9.054), ('听', 8.8439), ('计划', 8.8439), ('14', 8.8334), ('飞机', 8.8231)]


来重构之前的检索

如果没有索引，那就遍历每一条数据，将数据中关键字的 计算其tf并乘以IDF 然后求得该数据的最终得分。最后根据得分对数据进行排序。这样计算量很大。

现在我们这样，首先根据IDF对查询关键字进行排序，(然后过滤掉过低的关键字)
> 由于我们数据较小 多个关键字共现的数据没有 但当数据集特别大的时候 是会存在的 但依然可以使用此方法优化

再根据每个关键字索引得到包含关键字的数据子集，然后计算数据的得分，排序，返回结果。

In [84]:
# 假设数据子集依然是 ['房价','北京']的数据集
print('索引找到的数据子集：')
for d in finds:
    print(df.iloc[d].content)
print()

# 计算最后的得分：
tfidf_result = []
for d in finds:
    words = df.iloc[d].content.split()
    # 统计单词出现次数
    tf = dict(Counter(words))
    # 统计词频
    tf = {k:v/len(tf) for k,v in tf.items() if k in query}
    # 计算TFIDF
    tfidf = [v*idf[k] for k,v in tf.items()]
    [print(f'word:{k} tf:{v} idf:{idf[k]}') for k,v in tf.items()]
    print()
    tfidf_result.append(sum(tfidf))

# 打印排序结果
index = np.argmax(tfidf_result)
row_id = finds[index]
print(tfidf_result)
print(f'tfidf:{max(tfidf_result)}, data:{df.iloc[row_id].content}')

索引找到的数据子集：
房价 白跌 了 ！ 北京 首 套房 贷款 利率 上浮 10%
你 认为 北京 未来 5 年 的 房价 会 下跌 吗 ？
「 重磅 」 北京 限 房价 项目 销售 办法 出炉   部分 将 收作 共有 产权 房
北京 的 房价 还会 继续 上涨 吗 ？
北京 房贷利率 下周 再涨   专家 ： 累计 相当于 房价 上调 了 10%
北京 限价 房转 共有 产权 房 政策 拟 出炉   对 房价 有何 影响 ？
房价 不涨 卖 不动 ， 北京 首 套房 利率 继续 上浮 ， 刚需 招惹 了 谁 ？
北京 的 房价 还会 继续 上涨 吗 ？

word:房价 tf:0.09090909090909091 idf:6.9697
word:北京 tf:0.09090909090909091 idf:7.7348

word:北京 tf:0.08333333333333333 idf:7.7348
word:房价 tf:0.08333333333333333 idf:6.9697

word:北京 tf:0.0625 idf:7.7348
word:房价 tf:0.0625 idf:6.9697

word:北京 tf:0.125 idf:7.7348
word:房价 tf:0.125 idf:6.9697

word:北京 tf:0.08333333333333333 idf:7.7348
word:房价 tf:0.08333333333333333 idf:6.9697

word:北京 tf:0.07142857142857142 idf:7.7348
word:房价 tf:0.07142857142857142 idf:6.9697

word:房价 tf:0.0625 idf:6.9697
word:北京 tf:0.0625 idf:7.7348

word:北京 tf:0.125 idf:7.7348
word:房价 tf:0.125 idf:6.9697

[1.3367727272727272, 1.2253749999999999, 0.91903125, 1.8380625, 1.2253749999999999, 1.0503214285714284, 0.91903125, 1.8380625]
tfidf:1.8380625, d

## pagerank算法

[PageRank算法原理与实现](https://juejin.im/post/5b8ce27ef265da433f2a2f45)

## 搜索引擎相关项目：
- [jorendorff/tinysearch](https://github.com/jorendorff/tinysearch/blob/master/tiny.py)
- [trein/simple-search-engine](https://github.com/trein/simple-search-engine)

## 作业：复现

[狗屁不通文章生成器](https://github.com/menzi11/BullshitGenerator)